# AHP
評価基準：出塁率$c_1$, 長打率$c_2$,守備率$c_3$, 盗塁成功率$c_4$<br>
代替案：選手A$_1, a_1$, 選手A$_2, a_2$, 選手A$_3, a_3$<br> 

### 変数の名前の説明
mat : 2次元または3次元の配列であることを明示するための接頭語<br>
P   : 比較一対行列（Pairwise compare matrix）<br>
語尾のc : criteria （評価基準）に関連することを表す<br>
eigv : eigen-valueの略称

In [ ]:
# -*- coding: utf-8 -*- 
import numpy as np
import scipy.stats

import matplotlib.pyplot as plt
%matplotlib inline

評価基準に対する比較一対行列matPCで表す。<br>
各評価基準の項目（4つ）毎の比較一対行列を3次元配列matPにまとめることで，アルゴリズムの簡潔さを図る。<br>
変数L, Mは，それぞれの行列（正方行列であることに留意）の次元を表す。

In [ ]:
L = 4
matPc = np.array( [[1, 7, 1/3, 3], [1/7, 1, 1/5, 1/3], [3, 5, 1, 7], [1/3, 3, 1/7, 1]])
M = 3
matP1 = np.array( [[1, 1, 1/3], [1, 1, 1/5], [3, 5, 1]])
matP2 = np.array( [[1, 3, 7], [1/3, 1, 5], [1/7, 1/5, 1]])
matP3 = np.array( [[1, 5, 9], [1/5, 1, 5], [1/9, 1/5, 1]])
matP4 = np.array( [[1, 1/5, 3], [5, 1, 7], [1/3, 1/7, 1]])
matP = np.array([matP1, matP2, matP3, matP4]) 

## 固有値法
正方行列$\boldsymbol{P}$に対する固有値，固有ベクトルとの関係は次式で表される。
$$
\boldsymbol{P}\boldsymbol{w} = \lambda \boldsymbol{w}
$$
固有値，固有ベクトルを求める関数は次を使う<br>
numpy.linalg.eig: https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html

次の関数は，最大固有値とそれに対応する固有ベクトルを求め，固有ベクトルは要素の総和が1となるように正規化する。

In [ ]:
def getWeightVecEigen(mat):
    eig_val, eig_vec = np.linalg.eig(mat)
    abs_eig_val = np.abs(eig_val)  # In case of complex number
    idx = np.argmax(abs_eig_val)   # max eigenvalue
    max_eig_val = abs_eig_val[idx]
    abs_v = np.abs(eig_vec[:,idx]) # 
    return max_eig_val, abs_v/abs_v.sum()      # Normalization so that the sum is one. 

評価基準の一対比較行列matPcと，その最大固有値eigvmax_cと正規化した重みベクトルvcを求める。<br>

In [ ]:
eigvmax_c, v_c = getWeightVecEigen(matPc)
print("max of eigen value :",eigvmax_c)
print("its eigen vector:", v_c)

#### 評価基準の整合度（CI: Consistency Index）の確認
CI $\le 0.1 \sim 0.15$程度であれば良いとされている。<br>

In [ ]:
CI_c = (eigvmax_c - L)/(L-1)
print("CI_c=",CI_c)

#### 重要度ベクトルの棒グラフ化
List of named colors : https://matplotlib.org/stable/gallery/color/named_colors.html

In [ ]:
fig = plt.subplots(figsize=(6,3))
xtic = list(range(1,L+1))
plt.bar(x=xtic, height=v_c, width=0.3, color="b")
plt.xticks(xtic)
plt.xlabel('Criteria c_i', size=15)
plt.ylabel('Weight', size=15)
plt.grid()
#plt.savefig('fig_OR_AHP_Eigen_Weight_01.pdf', bbox_inches='tight')
plt.show()

各一対比較行列に対する最大固有値max_eigv[i]と正規化された重みベクトルv[i]を求める。さらに，CIの値を確認する。<br>
重みベクトルから重み行列matWを作る。

In [ ]:
v = np.zeros((L, M))
max_eigv = np.zeros(L)
for i in range(L):
    max_eigv[i], v[i] = getWeightVecEigen(matP[i])
    CI = (max_eigv[i] - M)/(M-1)
    print("CI=",CI, " max_eig_val=",max_eigv[i])

matW = v.T
matW

In [ ]:
matW[0]

In [ ]:
fig = plt.subplots(figsize=(8,4))
#x_1, x_2, x_3は棒グラフを並べてプロットするための棒グラフの位置
x_1 = [0.85, 0.95, 1.05, 1.15]
x_2 = [1.85, 1.95, 2.05, 2.15]
x_3 = [2.85, 2.95, 3.05, 3.15]

colors=["silver", "orangered", "royalblue", "lime"]
labels=["c1", "c2", "c3", "c4"]
xtic = list(range(1,M+1))

plt.bar(x=x_1, height=matW[0], width=0.1, color=colors, edgecolor="black")
plt.bar(x=x_2, height=matW[1], width=0.1, color=colors, edgecolor="black")
plt.bar(x=x_3, height=matW[2], width=0.1, color=colors, edgecolor="black")
plt.xticks(xtic)
plt.xlabel('Alternatives A_i', size=15)
plt.ylabel('Weight', size=15)
#plt.savefig('fig_OR_AHP_Eigen_Weight_02.pdf', bbox_inches='tight')
plt.grid()


#### 総合評価

In [ ]:
a = np.dot(matW, v_c)
print("a=",a)

これより，a[0] が最も良さそうであると言える。このグラフ化が次である。

In [ ]:
fig = plt.subplots(figsize=(6,3))
xtic = list(range(1,M+1))
plt.bar(x=xtic, height=a, width=0.3, color="cyan", edgecolor="black")
plt.xticks(xtic)
plt.xlabel('Alternatives A_i', size=15)
plt.ylabel('Total Score', size=15)
plt.grid()
#plt.savefig('fig_OR_AHP_Eigen_Weight_03.pdf', bbox_inches='tight')
plt.show()

## 幾何平均　（geometric mean）
$$
\bar{x}_G = \sqrt[n]{x_1 \times \cdots \times x_n}  = \left( \prod_{i=1}^{n} x_i  \right)^{\frac{1}{n}}
$$
比較一対行列の行方向に幾何平均をとり，ベクトルを得る。このベクトルを総和が１となる意味での正規化を行う。<br>
この計算はSciPyの次を使う。<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.gmean.html

In [ ]:
def getWeightVecGmean(mat):
    n, m = mat.shape
#    print('---', n, m)
    xg = np.zeros(n)
    for i in range(n):
        xg[i] = scipy.stats.gmean(mat[i])
#        print(i, "\n", mat[i], "\n", xg[i])
    return np.abs(xg)/np.abs(xg.sum())

評価基準の重みベクトルを得る。

In [ ]:
vg_c = getWeightVecGmean(matPc)
vg_c

評価基準の整合度（CI: Consistency Index）を確認する。CI  ≤0.1∼0.15 程度であれば良いとされている。<br>
vv = matPC vg_c を計算して，要素ごとの除算 vv/vg_cにより正規化して，全要素に対する平均を求める。<br>
これを最大固有値の推定値とする。

In [ ]:
vPvgc = np.dot(matPc, vg_c)
vdivc = np.divide(vPvgc, vg_c)
est_eigv = vdivc.mean()
CI_c = (est_eigv - L)/(L-1)
print(est_eigv, CI_c)

各一対比較行列に対し，幾何平均を用いた重みベクトルvg[i]を求める。<br>
さらに，CIの値を確認する。<br>
重みベクトルから重み行列matWを作る。

In [ ]:
vg = np.zeros((L, M))
for i in range(L):
    vg[i] = getWeightVecGmean(matP[i])
    vPvg = np.dot(matP[i], vg[i])
    vdiv = np.divide(vPvg, vg[i])
    est_eigv = vdiv.mean()
    CI = (est_eigv - M)/(M-1)
    print(CI, est_eigv)

matW = vg.T
matW

#### 総合評価

In [ ]:
ag = np.dot(matW, vg_c)
print("ag=",ag)

これより，ag[0] が最も良さそうであると言える。